In [1]:
import findspark
findspark.init("/opt/spark/")
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import regexp_replace
import configparser
import os
from pyspark.sql import functions as F
from pyspark.sql.functions import month, date_format
from pyspark.sql.functions import explode, col, from_json, struct
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import split, explode, regexp_replace
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

from pyspark.sql.functions import split, explode, regexp_replace, from_json

from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [2]:
spark = SparkSession.builder \
.appName("week10") \
.config("spark.jars.packages", "org.postgresql:postgresql:42.7.2") \
.master("local[2]") \
.getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/train/.ivy2/cache
The jars for the packages stored in: /home/train/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-49580479-2879-44d6-9c5a-d6bf777fe1a9;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.2 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 145ms :: artifacts dl 5ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------

In [3]:
! java --version


openjdk 11.0.22 2024-01-16 LTS
OpenJDK Runtime Environment (Red_Hat-11.0.22.0.7-1.el7_9) (build 11.0.22+7-LTS)
OpenJDK 64-Bit Server VM (Red_Hat-11.0.22.0.7-1.el7_9) (build 11.0.22+7-LTS, mixed mode, sharing)


In [4]:
df = spark.read \
.format("csv") \
.option("header", True) \
.option("inferSchema", True) \
.option("sep",",") \
.load("datasets/tmdb_5000_movies_and_credits/tmdb_5000_credits.csv")


In [5]:
df = spark.read \
.format("csv") \
.option("header", True) \
.option("inferSchema", True) \
.option("compression","gzip") \
.option("quote", "\"") \
.option("escape", "\"") \
.load("datasets/tmdb_5000_movies_and_credits/tmdb_5000_credits.csv")


In [6]:
df.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- crew: string (nullable = true)



In [7]:
df.show(2)

+--------+--------------------+--------------------+--------------------+
|movie_id|               title|                cast|                crew|
+--------+--------------------+--------------------+--------------------+
|   19995|              Avatar|[{"cast_id": 242,...|[{"credit_id": "5...|
|     285|Pirates of the Ca...|[{"cast_id": 4, "...|[{"credit_id": "5...|
+--------+--------------------+--------------------+--------------------+
only showing top 2 rows



In [8]:
#şema 
cast_schema = StructType([
    StructField("cast_id", IntegerType(), True),
    StructField("character", StringType(), True),
    StructField("credit_id", StringType(), True),
    StructField("gender", IntegerType(), True),
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("order", IntegerType(), True)
])
# cast sutunu JSON  
df = df.withColumn("cast", from_json(col("cast"), ArrayType(cast_schema)))
# cast sutunu explode
df = df.withColumn("cast", explode(col("cast")))


# credits_cast df
credits_cast = df.select(
    col("movie_id"),
    col("title"),
    col("cast.cast_id"),
    col("cast.credit_id"),
    col("cast.gender"),
    col("cast.id"),
    col("cast.name"))

# show
credits_cast.show(truncate=False)




+--------+------+-------+------------------------+------+-------+-------------------+
|movie_id|title |cast_id|credit_id               |gender|id     |name               |
+--------+------+-------+------------------------+------+-------+-------------------+
|19995   |Avatar|242    |5602a8a7c3a3685532001c9a|2     |65731  |Sam Worthington    |
|19995   |Avatar|3      |52fe48009251416c750ac9cb|1     |8691   |Zoe Saldana        |
|19995   |Avatar|25     |52fe48009251416c750aca39|1     |10205  |Sigourney Weaver   |
|19995   |Avatar|4      |52fe48009251416c750ac9cf|2     |32747  |Stephen Lang       |
|19995   |Avatar|5      |52fe48009251416c750ac9d3|1     |17647  |Michelle Rodriguez |
|19995   |Avatar|8      |52fe48009251416c750ac9e1|2     |1771   |Giovanni Ribisi    |
|19995   |Avatar|7      |52fe48009251416c750ac9dd|2     |59231  |Joel David Moore   |
|19995   |Avatar|9      |52fe48009251416c750ac9e5|1     |30485  |CCH Pounder        |
|19995   |Avatar|11     |52fe48009251416c750ac9ed|2   